# Federated PyTorch TinyImageNet Tutorial

This notebook is an example of Transfer Learning 

Custom DataLoader is used with OpenFL Python API

In [ ]:
#Install dependencies if not already installed
!pip install torch torchvision
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import glob
from torch.utils.data import Dataset, DataLoader
from PIL import Image

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms as T

import openfl.native as fx
from openfl.federated import FederatedModel, FederatedDataSet

After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [ ]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist')

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

#### Download the data

In [ ]:
!wget --no-clobber http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -n tiny-imagenet-200.zip
TINY_IMAGENET_ROOT = './tiny-imagenet-200/'

#### Describe the dataset

In [ ]:
class TinyImageNet(Dataset):
    """
    Contains 200 classes for training. Each class has 500 images. 
    Parameters
    ----------
    root: string
        Root directory including `train` and `val` subdirectories.
    split: string
        Indicating which split to return as a data set.
        Valid option: [`train`, `val`]
    transform: torchvision.transforms
        A (series) of valid transformation(s).
    """
    def __init__(self, root, split='train', transform=None, target_transform=None):
        NUM_IMAGES_PER_CLASS = 500
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.split_dir = os.path.join(self.root, split)
        self.image_paths = sorted(glob.iglob(os.path.join(self.split_dir, '**', '*.JPEG'), recursive=True))
        
        self.labels = {}  # fname - label number mapping

        # build class label - number mapping
        with open(os.path.join(self.root, 'wnids.txt'), 'r') as fp:
            self.label_texts = sorted([text.strip() for text in fp.readlines()])
        self.label_text_to_number = {text: i for i, text in enumerate(self.label_texts)}

        if split == 'train':
            for label_text, i in self.label_text_to_number.items():
                for cnt in range(NUM_IMAGES_PER_CLASS):
                    self.labels[f'{label_text}_{cnt}.JPEG'] = i
        elif split == 'val':
            with open(os.path.join(self.split_dir, 'val_annotations.txt'), 'r') as fp:
                for line in fp.readlines():
                    terms = line.split('\t')
                    file_name, label_text = terms[0], terms[1]
                    self.labels[file_name] = self.label_text_to_number[label_text]
                    
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        file_path = self.image_paths[index]
        label = self.labels[os.path.basename(file_path)]
        label = self.target_transform(label) if self.target_transform else label
        return self.read_image(file_path), label

    def read_image(self, path):
        img = Image.open(path)
        return self.transform(img) if self.transform else img

def one_hot(labels, classes):
    return np.eye(classes)[labels]

In [ ]:
normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

augmentation = T.RandomApply([
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.RandomResizedCrop(64)], p=.8)

training_transform = T.Compose([
    T.Lambda(lambda x: x.convert("RGB")),
    augmentation,
    T.ToTensor(),
    normalize])

valid_transform = T.Compose([
    T.Lambda(lambda x: x.convert("RGB")),
    T.ToTensor(),
    normalize])

#### Implement Federated dataset
We have to implement `split` method

In [ ]:
from openfl.utilities.data_splitters import EqualNumPyDataSplitter
from torch.utils.data import Subset


train_set = TinyImageNet(TINY_IMAGENET_ROOT, 'train', transform=training_transform)
valid_set = TinyImageNet(TINY_IMAGENET_ROOT, 'val', transform=valid_transform, \
                                      target_transform=lambda target: one_hot(target, 200))

class TinyImageNetFederatedDataset(DataLoader):
    def __init__(self, train_set, valid_set, batch_size):
        self.data_splitter = EqualNumPyDataSplitter()
        self.train_set = train_set
        self.valid_set = valid_set
        self.batch_size = batch_size
    
    def split(self, num_collaborators):
        train_split = self.data_splitter.split([label for _, label in self.train_set], num_collaborators)
        valid_split = self.data_splitter.split([label for _, label in self.valid_set], num_collaborators)
        return [
            TinyImageNetFederatedDataset(
                Subset(self.train_set, train_split[i]),
                Subset(self.valid_set, valid_split[i]),
                self.batch_size
            )
            for i in range(num_collaborators)
        ]
    
    def get_feature_shape(self):
        return self.train_set[0][0].shape
    
    def get_train_loader(self, num_batches=None):
        return DataLoader(self.train_set, batch_size=self.batch_size)
    
    def get_valid_loader(self):
        return DataLoader(self.valid_set)
    
    def get_train_data_size(self):
        return len(self.train_set)
    
    def get_valid_data_size(self):
        return len(self.valid_set)
    
fl_data = TinyImageNetFederatedDataset(train_set, valid_set, batch_size=32)

num_classes = 200

#### Define model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = torchvision.models.mobilenet_v2(pretrained=True)
        self.model.requires_grad_(False)
        self.model.classifier[1] = torch.nn.Linear(in_features=1280, \
                        out_features=num_classes, bias=True)

    def forward(self, x):
        x = self.model.forward(x)
        return x

    
optimizer = lambda x: optim.Adam(x, lr=1e-4)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.cross_entropy(input=output,target=target)

In [ ]:
#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy, \
                        data_loader=fl_data)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [ ]:
collaborator_models = fl_model.setup(num_collaborators=10)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

In [ ]:
#Original TinyImageNet dataset
print(f'Original training data size: {len(fl_data.train_set)}')
print(f'Original validation data size: {len(fl_data.valid_set)}\n')

#Collaborator one's data
for i, model in enumerate(collaborator_models):
    print(f'Collaborator {i}\'s training data size: {len(model.data_loader.train_set)}')
    print(f'Collaborator {i}\'s validation data size: {len(model.data_loader.valid_set)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

In [ ]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(collaborators,{'aggregator.settings.rounds_to_train':10})

In [ ]:
#Save final model
final_fl_model.save_native('final_model.pth')